In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt


### 1. Formulate the statement of the interpolation problem with Cubic Spline [mathematical formula]

Given a set of $n+1$ points defined on $[a, b]$ such that $a = x_0 < x_1 < ... < x_n = b$, the cubic spline $S(x)$ is a 
function, that satisfies:
- $S(x)$ is a polynomial of degree 3
- $S(x_i) = y_i $ for each interval $[x_i, x_{i+1}]$
- $S(x_i)$ has continuous derivatives of degrees $1$ to $p$ on $[a, b]

### 2. Formulate the functional and differential compatibility conditions [mathematical formula]

Functional compatibility: $S(x_i) = f(x_i) $ for $i = 0,1,...,n-1$  
Differential compatibility: $S^{(p)}(x_i) = f^{(p)}(x_i) $ for $i = 0,1,...,n-1$ and $p > 0$


### 3. Formulate stitching conditions [mathematical formula]

$S_{m, i-1}^{(p)}(x)|_{x = x_i} = S_{m, i}^{(p)}(x)|_{x = x_i}$
for $i = 1,2,...,n-1$ and $m = 3$ in our case

### 4. Justify why these conditions provide you with the required smoothness [thesis text, no more than 500 characters]

**Functional compatibility** ensures that derived function actually solves the interpolation problem.   
**Differential compatibility** ensures ensures that that derived function is differentiable up to $p$ degree.  
**Stiching conditions** ensures that the derived function is continuous.

### 5. Derive dependency formula: the dependence of the second derivatives at the grid nodes on the increment of the function (the function values difference on the grid nodes). [Mathematical formulas derivation. Detailed, with clear transitions]

Solve equation system composed of compatibility conditions 
to find $a_{0, i}, a_{1, i}, a_{2, i}, a_{3, i}$ 
for $i = 1, 2, ..., n-1$  
$\begin{cases}
S_{3, i}(x_i)-f(x_i)=0 \\
S_{3, i}(x_{i + 1})-f(x_{i + 1})=0 \\
S_{3, i}^{''}(x_i)-f^{''}(x_i)=0 \\
S_{3, i}^{''}(x_{i + 1})-f^{''}(x_{i + 1})=0
\end{cases}$ 
where $S_{i} = a_{0, i} + a_{1, i}(x - x_i) + a_{2, i}(x - x_i)^2 + a_{3, i}(x - x_i)^3$: 

Spline derivatives:   
$S^{'}_{3, i}(x) = a_{1,i} + 2a_{2, i}(x - x_i) + 3a_{3, i}(x - x_i)^2$  
$S^{''}_{3, i}(x) = 2a_{2, i} + 6a_{3, i} (x - x_i)$  
$S^{'''}_{3, i}(x) = 6a_{3, i} $

then

$\begin{cases}
a_{0, i} + a_{1, i}h_{i + 1} + a_{2, i}h_{i + 1}^2 + a_{3, i}h_{i + 1}^3 - f_{i + 1} = 0 \\
2a_{2, i} + 6a_{3, i}h_{i + 1} - m_{i + 1} = 0 \\
2a_{2, i} - m_i = 0 \\
a_{0, i} - f_i = 0 
\end{cases}$  
, where:  
$h_{i + 1} = x_{i + 1} - x_{i}$  
$\Delta m_i = m_{i + 1} - m_{i}$  
$\Delta f_i = f_{i + 1} - f_{i}$  

then

$a_{0, i} = f_i; a_{2, i} = \frac{m_i}{2} $  

$m_i + 6a_{3, i}h_{i + 1} - m_{i + 1} = 0 
\Rightarrow a_{3, i} = \frac{m_{i + 1} - m_i}{6h_{i + 1}}  = \frac{\Delta m_i}{6h_{i + 1}}$  

$f_i + a_{1, i}h_{i + 1} + \frac{m_i}{2}h_{i + 1}^2 + \frac{\Delta m_i}{6h_{i + 1}}h_{i + 1}^3 - f_{i + 1} = 0 
\Rightarrow -\frac{\Delta f_i}{h_{i + 1}} + a_{1, i} + \frac{m_i}{2}h_{i + 1} + \frac{\Delta m_i}{6}h_{i + 1} = 0 \Rightarrow$
$\Rightarrow a_{1, i} = \frac{\Delta f_i}{h_{i + 1}} - \frac{h_{i + 1}}{2}m_i - \frac{h_{i + 1}}{6}\Delta m_i$

thus

$S_{3, i} = f_i + 
(\frac{\Delta f_i}{h_{i + 1}} - \frac{h_{i + 1}}{2}m_i - \frac{h_{i + 1}}{6}\Delta m_i)(x - x_i) + 
\frac{m_i}{2}(x - x_i)^2 + \frac{\Delta m_i}{6h_{i + 1}}(x - x_i)^3$

$S_{3, i}^{'}(x_i) = \frac{\Delta f_i}{h_{i + 1}} - 
\frac{h_{i + 1}}{2}m_i - 
\frac{h_{i + 1}}{6}\Delta m_i$

and 

$S_{3, i - 1}(x) = f_{i - 1} + 
(\frac{\Delta f_{i - 1}}{h_{i}} - \frac{h_{i}}{2}m_{i - 1} - \frac{h_{i}}{6}\Delta m_{i - 1})(x - x_{i - 1}) + 
\frac{m_{i - 1}}{2}(x - x_{i - 1})^2 + \frac{\Delta m_{i - 1}}{6h_{i}}(x - x_{i - 1})^3$

$S_{3, i - 1}^{'}(x_i) = \frac{\Delta f_{i - 1}}{h_i} - 
\frac{h_i}{2}m_{i - 1} - 
\frac{h_i}{6}\Delta m_{i - 1} + 
h_im_{i - 1} + 
\frac{h_i}{2}\Delta m_{i - 1}$


From stitching condition $S_{3,i - 1}^{'}(x)|_{x=x_i} = S_{3,i}^{'}(x)|_{x=x_i}$, therefore:

$\frac{\Delta f_{i - 1}}{h_i} - \frac{h_i}{2}m_{i - 1} - \frac{h_i}{6}\Delta m_{i - 1} + h_im_{i - 1} + \frac{h_i}{2}\Delta m_{i - 1} = 
\frac{\Delta f_i}{h_{i + 1}} - \frac{h_{i + 1}}{2}m_i - \frac{h_{i + 1}}{6}\Delta m_i \Rightarrow$

$\Rightarrow \frac{h_i}{2}m_{i - 1} - \frac{h_i}{6}m_{i} + \frac{h_i}{6}m_{i - 1} + \frac{h_i}{2}m_{i} - \frac{h_i}{2}m_{i - 1} + \frac{h_{i + 1}}{2}m_i - \frac{h_{i + 1}}{6}m_{i + 1} + \frac{h_{i + 1}}{6}m_i = 
\frac{\Delta f_i}{h_{i + 1}} - \frac{\Delta f_{i - 1}}{h_i}$

thus

$\frac{h_i}{6}m_{i - 1} + \frac{h_i + h_{i + 1}}{3}m_i + \frac{h_{i + 1}}{6}m_{i + 1} = 
\frac{\Delta f_i}{h_{i + 1}} - \frac{\Delta f_{i - 1}}{h_i}$
for $i = 1, 2, ..., n-1$

### 6. Create a system of equations using this formula [Matrix representation. Mathematical formulas]

$\begin{bmatrix}
\frac{h_1}{6} & \frac{h_1+h_2}{3} & \frac{h_2}{6} & 0 & ... & ... & 0 \\
... & ... & ... & ... & ... & ... & ... \\ 
...& 0 & \frac{h_i}{6} & \frac{h_i+h_{i+1}}{3} & \frac{h_{i+1}}{6} & 0 & ... \\
... & ... & ... & ... & ... & ... & ... \\
0 & ... & ... & 0 & \frac{h_{n-1}}{6} & \frac{h_{n-1}+h_n}{3} & \frac{h_n}{6}
\end{bmatrix}
\begin{bmatrix}
m_0 \\ ... \\ m_i \\ ... \\ m_n
\end{bmatrix}
=
\begin{bmatrix}
\frac{\Delta f_1}{h_2}-\frac{\Delta f_0}{h_1} 
\\ ... \\
\frac{\Delta f_i}{h_{i+1}}-\frac{\Delta f_{i-1}}{h_i}
\\ ... \\
\frac{\Delta f_{n-1}}{h_n}-\frac{\Delta f_{n-2}}{h_{n-1}}
\end{bmatrix}$



### 7. Explain what is an unknown variable in this system. whether the system is closed with respect to an unknown variable. What is missing for closure. [Text, no more than 200 characters]

The unknown variable here is $m_i$ for $i = 0, 1, 2, ..., n$.  
The system is not closed since there is $n$ unknown variables, but only $n - 2$ equations for $i$ in $1$ to $n-1$.  
For closure we can add 2 extra equations of edge conditions: $m_0 = 0$ and $m_n = 0$ 

### 8. Bring this matrix to the appropriate form to use the Tridiagonal matrix algorithm [Mathematical derivation. Use Gauss Elimination]

$\begin{bmatrix}
1 & -P_1 & 0 & 0 & ... & Q_1 \\
0 & 1 & -P_2 & 0 & ... & Q_2 \\
0 & 0 & 1 & -P_3 & ... & Q_3 \\
... & ... & ... & ... & ... & ... \\
0 & 0 & 0 & 1 & ... & Q_{n-1} \\
\end{bmatrix}$  
  
Direct:  
$P_i = \frac{h_{i + 1}}{-2(h_{i+1} + h_i) - h_i P_{i-1}}$
, 
$P_1 = \frac{h_2}{-2(h_2 + h_1)}$  

$Q_i = \frac
{h_i Q_{i - 1} - 6(\frac{\Delta f_i}{h_{i + 1}} - \frac{\Delta f_{i - 1}}{h_i})}
{-2(h_{i + 1} + h_i) - h_i P_{i - 1}}$
, 
$Q_1 = \frac{6(\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1})}{2(h_2 + h_1)}$

Reverse:  
$x_i = P_i x_{i+1} + Q_i$, $x_{n - 1} = Q_{n - 1}$

### 9. Derive formulas of direct pass and reverse pass of Tridiagonal matrix algorithm [Mathematical formals]

After conversion of tridiagonal matrix from question 6 to upper triangular matrix from question 8 we can see that each
row depends on the next row, and from last row we see that $x_n = Q_n$, thus we can make recursive formula 
for reverse pass:   
$x_i = P_i x_{i+1} + Q_i$

A tridiagonal matrix can be represented as:  
$\alpha_ix_{i - 1} - \beta_ix_i + \gamma_ix_{i + 1} = \delta_i, \alpha_1=\gamma_n=0, i=1..n$

$x_i = P_i x_{i+1} + Q_i$, and therefore  
$x_{i-1} = P_{i-1} x_i + Q_{i-1}$  

thus

$\alpha_i(P_{i - 1}x_i + Q_{i - 1}) - \beta_ix_i + \gamma_ix_{i + 1} = \delta_i \Rightarrow$  
$\Rightarrow x_i = 
\frac{-\gamma_i}{\alpha_iP_{i - 1} - \beta_i}x_{i + 1} + 
\frac{-\alpha_iQ_{i - 1} + \delta_i}{\alpha_iP_{i - 1} - \beta_i} \Rightarrow 
x_i = P_i x_{i+1} + Q_i$

thus the direct pass formulas are:  
$P_i = \frac{\gamma_i}{\beta_i - \alpha_iP_{i - 1}}$  
$Q_i = \frac{\alpha_iQ_{i - 1} - \delta_i}{\beta_i - \alpha_iP_{i - 1}}$


### 10. Implement code prototype of the future algorithm implementation. Classes/methods (if you use OOP), functions. The final implementation (on language chosen by you) should not differ from the functions declared in the prototype. [Python code]

In [ ]:
### YOUR CODE HERE










### 11. Derive formula of Cubic Spline method error [Mathematical formulas]

In [ ]:
### YOUR TEXT/CODE HERE










### 12. Rate the complexity of the algorithm [Text, and rate in terms of big O, no more than 100 characters]

In [ ]:
### YOUR TEXT/CODE HERE










### Congrats!